<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Py_trodes/read_XYpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import struct
import numpy as np

def extract_position_time(file_loc):
    """
    Extracts time and position data from a binary file.
    Note: videoPositionTracking are binaries but are not
    labeled as such.

    Parameters:
    file_dir (str): The directory containing the file.
    filename (str): The name of the file to be read.

    Returns:
    tuple: A tuple containing two numpy arrays:
        - time_data: An array of time data.
        - position_data: An array of position data.
    """
    format_dict = {'uint32': 'I', 'uint16': 'H'}

    with open(file_loc, mode='rb') as file:
        filecontent = file.read()

    header = str(filecontent[:200])
    header = header.split('<End settings>')[0]
    field_section = header.split('Fields: ')[-1]
    fields = re.findall(r'<.*?>', field_section)
    units = [f.split(' ')[-1][:-1] for f in fields]
    field_names = [f.split(' ')[0][1:] for f in fields]
    record_format = ''.join([format_dict[u] for u in units])

    header_len = len(header + '<End settings>') - header.count('\\n') - 1
    record_format = ''.join([format_dict[u] for u in units])
    record_size = struct.calcsize(record_format)
    records = []

    with open(file_loc, 'rb') as file:
        file.seek(header_len)
        while chunk := file.read(record_size):
            if len(chunk) == record_size:
                record = struct.unpack(record_format, chunk)
                records.append(record)

    records = np.stack(records)
    time_data = records[:, 0]
    position_data = records[:, 1:]

    return time_data, position_data



"""
TEST
file_loc = r"\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\ZL04_6201\ZL04_6201.1.videoPositionTracking"
"""